<a href="https://colab.research.google.com/github/Rapheal-Jones/Image-segementation/blob/master/Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import os
import cv2
import numpy as np

In [ ]:
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Activation,Add,BatchNormalization,Input
from tensorflow.keras.layers import GlobalAveragePooling2D,AveragePooling2D,UpSampling2D,concatenate,Reshape,LeakyReLU

In [ ]:
device_name = tf.test.gpu_device_name()
device_name

In [ ]:
def images(path):
    i=0
    image,label=[],[]
    folder_file=os.listdir(os.path.join(path,dir[0]))

    for img in folder_file[:1000]:
        
        img_path=os.path.join(path,dir[0],img)
        lab_path=os.path.join(path,dir[1],img)
        
        img_arr=cv2.imread(img_path,cv2.IMREAD_UNCHANGED)
        lab_arr=cv2.imread(lab_path,cv2.IMREAD_COLOR)

        img_normalize=cv2.normalize(img_arr,None,norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        lab_normalize=cv2.normalize(lab_arr,None,norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        image.append(img_normalize)
        label.append(lab_normalize)
        
        i+=1
        print(i,end=' ')

    return np.array(image),np.array(label)

In [ ]:

dir=['image','label']

train_folder = "/content/drive/My Drive/Project/datasets/train/"
#test_folder = "/content/drive/My Drive/Project/datasets/test/"
#validate_folder = "/content/drive/My Drive/Project/datasets/val/"


image_train, label_train = images(train_folder)
# image_test,label_test = images(test_folder)
#image_val, label_val = images(validate_folder)

In [ ]:
def block_1(layer,filter1,filter2,filter3,block):

    layer = Conv2D(filter1,kernel_size=(1,1),dilation_rate=(1,1),padding='same',name='block_1_conv_1'+block)(layer)
    layer = BatchNormalization(name='block_1_BN_1'+block)(layer)
    layer = LeakyReLU(alpha=0.2,name='block_1_Acti_1'+block)(layer)

    layer = Conv2D(filter2,kernel_size=(3,3),dilation_rate=(2,2),padding='same',name='block_1_conv_2'+block)(layer)
    layer = BatchNormalization(name='block_1_BN_2'+block)(layer)
    layer = LeakyReLU(alpha=0.2,name='block_1_Acti_2'+block)(layer)

    layer = Conv2D(filter3,kernel_size=(1,1),dilation_rate=(1,1),padding='same',name='block_1_conv_3'+block)(layer)
    layer = BatchNormalization(name='block_1_BN_3'+block)(layer)

    #layer = MaxPool2D(pool_size=(2,2),name='block_1_pool'+block)(layer)

    return layer

In [ ]:
def block_2(layer,filter,block):

    layer = Conv2D(filter,kernel_size=(3,3),dilation_rate=(1,1),padding='same',name='block_2_conv'+block)(layer)
    layer = BatchNormalization(name='block_2_BN'+block)(layer)
    #layer = MaxPool2D(pool_size=(2,2),name='block_2_pool'+block)(layer)

    return layer

In [ ]:
def feature_map_block(layer,filters,block):

    block_1_output = block_1(layer,filters[0],filters[1],filters[2],block)
    block_2_output = block_2(layer,filters[2],block)
              
    layer = Add(name='feature_map_block_add'+block)([block_1_output,block_2_output])
    layer = Activation(activation='relu',name='feature_map_block_Acti'+block)(layer)

    return layer

In [ ]:
def feature_map(layer):

    layer = feature_map_block(layer,[32,32,64],'_a')
    layer = feature_map_block(layer,[64,64,128],'_b')
    layer = feature_map_block(layer,[128,128,3],'_c')

    return layer

In [ ]:
def pyramid_pooling(layer):

    features = feature_map(layer)

    red = GlobalAveragePooling2D()(features)
    red = Reshape((1,1,256))(red)
    red = Conv2D(64,kernel_size=(3,3),padding='same',dilation_rate=(1,1),name='red_conv')(red)

    orange = AveragePooling2D(pool_size=(2,2))(features)
    orange = Conv2D(64,kernel_size=(3,3),padding='same',dilation_rate=(1,1),name='orange_conv')(orange)

    blue = AveragePooling2D(pool_size=(4,4))(features)
    blue = Conv2D(64,kernel_size=(3,3),padding='same',dilation_rate=(1,1),name='blue_conv')(blue)

    green = AveragePooling2D(pool_size=(8,8))(features)
    green = Conv2D(64,kernel_size=(3,3),padding='same',dilation_rate=(1,1),name='green_conv')(green)

    return red,orange,blue,green,features

In [ ]:
def upsampling(layer):

    red, orange, blue, green, features= pyramid_pooling(layer)

    red = UpSampling2D(size=512,interpolation='bilinear',name='red_upsampling')(red)
    orange = UpSampling2D(size=16,interpolation='bilinear',name='orange_upsampling')(orange)
    blue = UpSampling2D(size=32,interpolation='bilinear',name='blue_upsampling')(blue)
    green= UpSampling2D(size=64,interpolation='bilinear',name='green_upsampling')(green)

    features = UpSampling2D(size=8,interpolation='bilinear',name='features_upsampling')(features)

    return concatenate([features,red,orange,blue,green],name='concatenate')

In [ ]:
def last_layer(layer):

    concat_layer = upsampling(layer)

    final_layer = Conv2D(3,kernel_size=(3,3),padding='same',name='final_conv')(concat_layer)
    final_layer = BatchNormalization(name='final_BN')(final_layer)
    final_layer = Activation('softmax',name='final_acti')(final_layer)

    return final_layer

In [ ]:
input_layer=Input(shape=(512,512,3),name='Input_layer')
output_layer=feature_map(input_layer)

model=Model(inputs=input_layer,outputs=output_layer)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(image_train,label_train,epochs=10,batch_size=2)